In [2]:
# # Mounting GCS to colab
# # https://stackoverflow.com/questions/51715268/how-to-import-data-from-google-cloud-storage-to-google-colab

# from google.colab import auth
# auth.authenticate_user()

In [5]:
# !echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" > /etc/apt/sources.list.d/gcsfuse.list
# !curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -
# !apt -qq update
# !apt -qq install gcsfuse

In [6]:
# !mkdir colab_indoor
# !gcsfuse indoor-data colab_indoor
# !mkdir colab_indoor/train_4
# !gcsfuse indoor-data/train_4 colab_indoor/train_4
# !mkdir colab_indoor/test_4
# !gcsfuse indoor-data/test_4 colab_indoor/test_4

In [ ]:
# !pip install torchviz

In [7]:
!ls -la -h ./colab_indoor/train_4

total 4.2G
-rw-r--r-- 1 root root 158M Apr 10 23:19 5a0546857ecc773753327266_train.csv
-rw-r--r-- 1 root root 150M Apr 10 23:20 5c3c44b80379370013e0fd2b_train.csv
-rw-r--r-- 1 root root 812M Apr 10 23:20 5d27075f03f801723c2e360f_train.csv
-rw-r--r-- 1 root root 224M Apr 10 23:20 5d27096c03f801723c31e5e0_train.csv
-rw-r--r-- 1 root root 135M Apr 10 23:20 5d27097f03f801723c320d97_train.csv
-rw-r--r-- 1 root root  23M Apr 10 23:20 5d27099f03f801723c32511d_train.csv
-rw-r--r-- 1 root root  28M Apr 10 23:20 5d2709a003f801723c3251bf_train.csv
-rw-r--r-- 1 root root 154M Apr 10 23:20 5d2709b303f801723c327472_train.csv
-rw-r--r-- 1 root root 216M Apr 10 23:20 5d2709bb03f801723c32852c_train.csv
-rw-r--r-- 1 root root 289M Apr 10 23:20 5d2709c303f801723c3299ee_train.csv
-rw-r--r-- 1 root root 112M Apr 10 23:20 5d2709d403f801723c32bd39_train.csv
-rw-r--r-- 1 root root  80M Apr 10 23:20 5d2709e003f801723c32d896_train.csv
-rw-r--r-- 1 root root 8.9M Apr 10 23:20 5da138274db8ce0c98bbd3d2_train.csv
-

In [8]:
import random
from random import sample
import numpy as np
import pandas as pd
import gc
import matplotlib.pyplot as plt
from scipy.ndimage.filters import uniform_filter1d
from scipy.interpolate import interp1d
import time
from collections import defaultdict
import seaborn as sns

import scipy.stats as stats
from pathlib import Path
import glob
import pickle
from tqdm import tqdm
import random
import os
import copy
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import multiprocessing


NUM_CORES = multiprocessing.cpu_count()
print(NUM_CORES)

OUTPUT_NAME = "train_8"

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

set_seed()

2


---
## Train 8
---

In [9]:
# train paths and test paths
train_files = sorted(glob.glob("./colab_indoor/train_4/*"))
test_files = sorted(glob.glob("./colab_indoor/test_4/*"))

# load submission file
sub_df = pd.read_csv("./colab_indoor/sample_submission.csv", index_col=0)
# sub_df[["site", "file", "timestamp"]] = sub_df["site_path_timestamp"].apply(lambda x: pd.Series(x.split("_")))
display(sub_df.head())

,floor,x,y
site_path_timestamp,,,
5a0546857ecc773753327266_046cfa46be49fc10834815c6_0000000000009,0,75.0,75.0
5a0546857ecc773753327266_046cfa46be49fc10834815c6_0000000009017,0,75.0,75.0
5a0546857ecc773753327266_046cfa46be49fc10834815c6_0000000015326,0,75.0,75.0
5a0546857ecc773753327266_046cfa46be49fc10834815c6_0000000018763,0,75.0,75.0
5a0546857ecc773753327266_046cfa46be49fc10834815c6_0000000022328,0,75.0,75.0


In [11]:
# Load train csv and test csv
train_df = pd.read_csv(train_files[0])
test_df = pd.read_csv(test_files[0])
display(train_df.head())
display(test_df.head())

,wifi_ts,wps_diff,x,y,floor,floor_int,file_id,site_id,acce_1000_x_avg,acce_1000_y_avg,acce_1000_z_avg,acce_2000_x_avg,acce_2000_y_avg,acce_2000_z_avg,acce_5000_x_avg,acce_5000_y_avg,acce_5000_z_avg,magn_1000_x_avg,magn_1000_y_avg,magn_1000_z_avg,magn_1000_st_avg,magn_2000_x_avg,magn_2000_y_avg,magn_2000_z_avg,magn_2000_st_avg,magn_5000_x_avg,magn_5000_y_avg,magn_5000_z_avg,magn_5000_st_avg,ahrs_1000_x_avg,ahrs_1000_y_avg,ahrs_1000_z_avg,ahrs_2000_x_avg,ahrs_2000_y_avg,ahrs_2000_z_avg,ahrs_5000_x_avg,ahrs_5000_y_avg,ahrs_5000_z_avg,gyro_1000_x_avg,gyro_1000_y_avg,...,6f311690978e199e95ad23003eb1deb1d0b2f42b,7e44cc6c807fe154ba7fe97bca97030f6e610d2b,db2118bbf6034e55c8a594481aa87d6a67e3bfbb,d6f8e0a20a1a65b0ce10a3d58c3f3a8758a95e6f,f6b4a651fe2db1c68c3652da07f7869453adfee8,52991874f27e29074381dc2b6cf4267ba079564b,85f187bfc2e3a1eab8027c925690679c1de6b5c7,4ed61e923a531cd500928e7dcf4412d5dabdd4e6,8b59fb25bf867e2b57ba14cb71c386d202d714df,3e30d40b1f40522801cbb68a468b92414f1ccf13,98a30c53639727fd1f43c9ab4237c3c3f6153ada,aecd9cc3d0c1640e971226c67678032d65747459,1ae65ecc64fd68a95580a4d968f9c9f8cf777ae9,b71b9159ec25bd67017caa99494263086a5cd326,c26c54c0dfa013b4ee1e15a3f11bcfa8215c0a99,1b12eb877cf5261ab74957f2ca659ee89e858a42,6e53995eb99731c76e4804cb193e160d60e03cff,21d6b5544a59eac82f9bb8bf54105a41958adaf8,94f0be14f76dfbb65a8d9c920542f06e4f79fa26,a630919f315899ebf8650eb07d1097204ba5a2d3,6471b2899033a43f16bfd62b8198ef9557a49f62,43b75ab26009a503bb7cc0b4776edc6c27621ea7,27f2fa8a8809bc430be43ea21c8968ed8f98cef7,48be90ff375acd4d6a323e4d85ede7564cebd98a,380da08a7279a1947c76c0961bf2c7bfa8bb2eaf,b2ce01476d4f985ee9d0518d1eadf9ed18d236ad,9a08f3c0cf309d67555c8f9c6f64686b9ba2283d,16f0057b4591b780054a4f2867ba3fa9ec793438,2e985809d94e7cfe3dbdc78a27e17baa56bef58b,1ad3144a286d2b32af03f8e275191f8c48c77e10,8c1eceb10a2c1b8d943f836acf2444392250f685,2a1bd89115889eacd8ef93c0fa2584e238a098a8,d16001127911cd8aad3e6e09663802d8748cd68e,e95654c4ebc092ae1845ba174839a8fc0e5c92b4,35e02baaefd4e7c076667ecadce166c2d2f09977,d90ab0610e02c2adb1174feaf719b9e506564893,01ead5ee97d0329b2c228fece5a96b0358a3cab1,8c077f9959ff86dc9f8836444618c2483f12e1ba,31595a6c2bd3a944b144685a2f117715520fd417,ecb87e6c93dd3c6cdb6f6671e25e2d0d6f865875
0,1578470975068,1913,141.92305,82.461280,B1,-1,5e158ecff4c3420006d52164,5a0546857ecc773753327266,-0.642224,2.972192,9.169450,-0.635654,3.132576,9.345822,-0.501901,3.065682,9.241528,-10.382296,32.349018,-52.060368,52.060368,-8.841762,34.115211,-52.633878,52.633878,-10.723718,27.716033,-50.914708,50.914708,0.197387,0.017648,-0.093118,0.197238,0.014744,-0.094190,0.196257,0.001104,-0.115151,-0.071993,-0.033518,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
1,1578470976967,3812,141.92305,82.461280,B1,-1,5e158ecff4c3420006d52164,5a0546857ecc773753327266,-0.228063,1.450971,9.934439,-0.375506,2.136542,9.585445,-0.787188,3.555897,9.001054,-12.904959,37.001978,-49.407959,49.407959,-14.127820,31.718291,-49.493185,49.493185,-10.349463,22.741579,-51.205094,51.205094,0.107017,-0.009636,-0.133157,0.148128,-0.005261,-0.133062,0.225686,0.008288,-0.126210,0.021126,-0.021466,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
2,1578470978867,3120,142.00758,87.245865,B1,-1,5e158ecff4c3420006d52164,5a0546857ecc773753327266,-0.373989,3.449454,9.115677,-0.634849,3.250454,9.074398,-0.794006,3.792492,8.980378,-12.806917,16.988458,-48.160237,48.160237,-12.733321,17.643661,-49.472755,49.472755,-10.725843,20.591118,-50.981995,50.981995,0.219273,-0.014600,-0.137240,0.215769,-0.006632,-0.155545,0.230807,0.003506,-0.136715,0.228284,0.010250,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-9

,site_path_timestamp,correct_wps_ts,wifi_ts,wps_diff,x,y,floor,floor_int,file_id,site_id,acce_1000_x_avg,acce_1000_y_avg,acce_1000_z_avg,acce_2000_x_avg,acce_2000_y_avg,acce_2000_z_avg,acce_5000_x_avg,acce_5000_y_avg,acce_5000_z_avg,magn_1000_x_avg,magn_1000_y_avg,magn_1000_z_avg,magn_1000_st_avg,magn_2000_x_avg,magn_2000_y_avg,magn_2000_z_avg,magn_2000_st_avg,magn_5000_x_avg,magn_5000_y_avg,magn_5000_z_avg,magn_5000_st_avg,ahrs_1000_x_avg,ahrs_1000_y_avg,ahrs_1000_z_avg,ahrs_2000_x_avg,ahrs_2000_y_avg,ahrs_2000_z_avg,ahrs_5000_x_avg,ahrs_5000_y_avg,ahrs_5000_z_avg,...,ead4eb320daf267710b8ec2f80b572ef14ca55f6,a19dac7dbdfd5b1628917159bb0333d0b4248b01,a5fa56c01bc636432ad0deb09ac769dd8b4bc4c5,c3403babce8901fd7a22f2752ee27e55c93137ef,e8af39b7dfe78bd5413b4af4a667d052e8cbaa77,fb91245a09b92f24ae14fc6e76e1b3549bd6a31d,0697d08b2741a320bcdcbb9624caca0e3a415356,2d199d76f05af1ff3432955b6b52163e8341dda5,b37d43c8c5f1e82765b5a403c19529d89b765d62,0076ff7a084cb2ac8c146139965ab1be296e72c4,b896a2a793aa802db868a7a0fdb29204bde99065,108716739163fd1a6e473359330704f7cd2068f7,e32d9e78dd87486b832f7723c916ca93ad686a54,1ad3144a286d2b32af03f8e275191f8c48c77e10,38f586d6a54a75a4fc9df0325de877b2bce5de6b,097599fa8b0d2bb4426ba0dae5ca9e578cdb8149,dc522df53e28d76a729dc56f81b1295a50ce4289,042e55ae2e3aae88070d507977eb97cd7468ef80,3b61536817d546393995f664b446e4ecf609ed61,295dcf2d00e2d19d02f73fc1d498ed5ca821161c,647649398bea2985feed4728d748939666dfb609,f3d9e7d629e50b8ca7e330d1d7cc11d6a75def8f,c70a6973225a173d7a0f74ca0ff93d07d140ed9d,a9f2d89eb253ffaf137bad7322e7cae41cd575d7,da8b535fe9eb7c6ea9388d1d2c4c80777da0fa2f,315dc4970761f38035d1da2d3592624abf1293ab,cb7c0d32ada93bf41f44aa2b37a5c584e894e040,96524d4078bf26b236a6ccc4efcf8bfb14189ab4,72a8a758661a41895e424613d666109e838b3d5f,af03d645e2ce58455848037f78dc79d962ec95d2,b831c303c1abd40fd8b242f902b6d900bca585af,c87c9fe93ddb4dc25d1d241c87a340ecf5be19f5,6bcdb1157c0d47e8a75d4ae864210156ebc2de96,817b99c65c4d92ab758443590680da2c3129c62b,563c56cbd1eba74106d9d4cf3adef19ddd574858,24f7fe5fa290f803c49bac656c57df514b180b4b,7e642dee5535c87ae5d65d225ee5dab01769b04c,ae2a68d930d314c1c164485b9ac5ef5beb158290,c5de94605289a9fae1869b9c73dde7806c559717,ed61d67ab40541ebb671536a7ea20125119cb964
0,5a0546857ecc773753327266_046cfa46be49fc1083481...,1578474564146,1578474566477,2331,-999.0,-999.0,-999.0,-999.0,046cfa46be49fc10834815c6,5a0546857ecc773753327266,-0.537289,3.761366,8.712986,-0.600666,3.619264,9.188262,-0.672788,3.816067,9.062869,28.431178,-1.460127,-36.664404,36.664404,27.825088,0.309160,-36.920043,36.920043,27.073482,-4.207470,-35.674299,35.674299,0.199123,0.134203,0.484158,0.181839,0.123274,0.484177,0.182630,0.141195,0.513942,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
1,5a0546857ecc773753327266_046cfa46be49fc1083481...,1578474573154,1578474573645,491,-999.0,-999.0,-999.0,-999.0,046cfa46be49fc10834815c6,5a0546857ecc773753327266,-0.542840,3.291618,9.183646,-0.855302,3.816327,9.127775,-0.878020,3.919553,8.960652,13.715247,6.633566,-31.755220,31.755220,12.138475,4.916706,-29.992745,29.992745,9.667867,3.765783,-32.982411,32.982411,0.160077,0.098560,0.268932,0.164551,0.111172,0.242563,0.188104,0.094040,0.210174,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
2,5a0546857ecc773753327266_046cfa46be49fc1083481...,1578474579463,1578474578851,612,-999.0,-999.0,-999.0,-999.0,046cfa46be49fc10834815c6,5a0546857ecc773753327266,-1.965967,3.984560,9.071354,-1.254228,3.870698,9.113594,-1.172762,3.916836,8.943026,-5.057949,12.982332,-40.837066,40.837066,-6.820432,14.723760,-39.435893,39.435893,-5.141440,13.560757,-37.624316,37.624316,0.244854,0.040413,-0.199164,0.228625,0.012472,-0.181569,0.224807,0.015700,-0.142028,...,-999,-999,-999,-999,-99

In [ ]:
# check nan ratio for wifi columns
# 3-4 min for 1 df -> ~1 hour for all df
# train columns or test columns? at least we have to work with overlapping ones only

dfs = []
for i, path in enumerate(train_files):
    train_df = pd.read_csv(path)
    test_df = pd.read_csv(test_files[i])

    # Match train and test columns
    all_train_cols = list(train_df.columns)
    all_test_cols = list(test_df.columns)
    print("all train cols: ", len(all_train_cols), "all test cols: ", len(all_test_cols))

    # get all non-overlapping columns
    no_overlap_col = list(set(all_train_cols) ^ set(all_test_cols))
    train_cols = [x for x in all_train_cols if x not in no_overlap_col]

    # filter out the df by the columns to leave
    train_df = train_df[train_cols]
    print("train_df cols: ", len(train_df.columns))
    dfs.append(train_df)

In [20]:
# calculate nan ratio for all columns
null_col_ratios = []
for df in dfs:
    cols = df.iloc[:, 50:].columns
    site_id = df.iloc[0, 7]
    print(site_id)
    for col in cols:
        null_count = len(df[df[col] == -999])
        null_ratio = null_count / len(df) * 100
        null_col_ratios.append([site_id, col, null_ratio])

null_col_df = pd.DataFrame(null_col_ratios, columns=["site_id", "column", "null_ratio"])

all train cols:  3456 all test cols:  2486
train_df cols:  2448
all train cols:  3122 all test cols:  1129
train_df cols:  1126
all train cols:  7088 all test cols:  1254
train_df cols:  1249
all train cols:  5023 all test cols:  1594
train_df cols:  1088
all train cols:  2549 all test cols:  1199
train_df cols:  1173
all train cols:  984 all test cols:  616
train_df cols:  611
all train cols:  1311 all test cols:  946
train_df cols:  927
all train cols:  1972 all test cols:  1227
train_df cols:  1204
all train cols:  2511 all test cols:  1615
train_df cols:  1574
all train cols:  5890 all test cols:  4424
train_df cols:  4380
all train cols:  2198 all test cols:  1827
train_df cols:  1660
all train cols:  1368 all test cols:  1060
train_df cols:  1012
all train cols:  549 all test cols:  359
train_df cols:  349
all train cols:  2921 all test cols:  1898
train_df cols:  1838
all train cols:  1271 all test cols:  968
train_df cols:  955
all train cols:  881 all test cols:  582
train_df 

In [21]:
null_col_df.to_csv("./colab_indoor/null_col_df.csv", index=False)
print(train_files[0])
display(null_col_df.head())

./colab_indoor/train_4/5a0546857ecc773753327266_train.csv


,site_id,column,null_ratio
0,5a0546857ecc773753327266,fd4804b1282054b5df1bc478fbcb73fbd1aa09cb,97.536575
1,5a0546857ecc773753327266,2253c6a0d0f7277737aa8e86e0484be805124806,95.600258
2,5a0546857ecc773753327266,2daa7fde27c0fd44e4a0133c3247be06d3c21c0b,96.815835
3,5a0546857ecc773753327266,ce0718d609ae0f06031490c462fff571210d0d0c,97.095525
4,5a0546857ecc773753327266,82385a56f25847fe6966d46bb58f302c4597953c,97.697935


In [ ]:
# Check wifi datas
print(train_files[0])
check_path = "./colab_indoor/train_4/5d2709bb03f801723c32852c_train.csv"
check_df = pd.read_csv(check_path)
check_df = check_df[check_df["file_id"] == "5dca6fe55e083300061bcab7"]
print(len(check_df))

./colab_indoor/train_4/5a0546857ecc773753327266_train.csv
207


In [ ]:
# Match train and test columns
all_train_cols = list(train_df.columns)
all_test_cols = list(test_df.columns)
print("all train cols: ", len(all_train_cols), "\n", "all test cols: ", len(all_test_cols))

# get all non-overlapping columns
no_overlap_col = list(set(all_train_cols) ^ set(all_test_cols))
no_overlap_col += ["wifi_ts", "floor", "file_id", "site_id"] # add other columns to exclude
train_cols = [x for x in all_train_cols if x not in no_overlap_col]
test_cols = [x for x in all_test_cols if x not in no_overlap_col]
# test_cols += ["site_path_timestamp"] # test_df needs to keep "site_path_timestamp"

# filter out the df by the columns to leave
train_df = train_df[train_cols]
test_df = test_df[test_cols]

# # Drop some columns not necessary as a feature
# drop_cols = ["wifi_ts", "floor", "file_id", "site_id"]
# for df in [train_df, test_df]:
#     df = df.drop(columns=drop_cols, inplace=True)

# Convert df object columns to integers and then the whole thing to tensors
for df in [train_df, test_df]:
    obj_col = list(df.select_dtypes(include=['object']).columns)
    for col in obj_col:
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col].values)

# Apply MinMaxScaler to each IMU columns
current_cols = list(train_df.columns)
imu_cols = current_cols[:23]
exception_columns = ["wps_diff", "x", "y", "floor_int", "rel_diff", "rel_x", "rel_y"]
imu_cols = [x for x in imu_cols if x not in exception_columns]
print("imu_cols: ", imu_cols)
for col in imu_cols:
    ss_scaler = StandardScaler()
    train_df[col] = ss_scaler.fit_transform(train_df[col].values.reshape(-1, 1))
    test_df[col] = ss_scaler.transform(test_df[col].values.reshape(-1, 1))

print(len(train_df.columns))
print(len(test_df.columns))
print(len(train_df))
print(len(test_df))
print("object dtype columns in train", train_df.select_dtypes(include=['object']).columns)
print("object dtype columns in test", test_df.select_dtypes(include=['object']).columns)
display(train_df.head())
display(test_df.head())

all train cols:  3424 
 all test cols:  2453
imu_cols:  ['acce_x_avg', 'acce_y_avg', 'acce_z_avg', 'magn_x_avg', 'magn_y_avg', 'magn_z_avg', 'magn_st', 'ahrs_x_avg', 'ahrs_y_avg', 'ahrs_z_avg', 'gyro_x_avg', 'gyro_y_avg', 'gyro_z_avg', 'magn_u_x_avg', 'magn_u_y_avg', 'magn_u_z_avg']
2412
2412
9296
300
object dtype columns in train Index([], dtype='object')
object dtype columns in test Index([], dtype='object')


,wps_diff,x,y,floor_int,acce_x_avg,acce_y_avg,acce_z_avg,magn_x_avg,magn_y_avg,magn_z_avg,magn_st,ahrs_x_avg,ahrs_y_avg,ahrs_z_avg,gyro_x_avg,gyro_y_avg,gyro_z_avg,magn_u_x_avg,magn_u_y_avg,magn_u_z_avg,rel_diff,rel_x,rel_y,fd4804b1282054b5df1bc478fbcb73fbd1aa09cb,2253c6a0d0f7277737aa8e86e0484be805124806,2daa7fde27c0fd44e4a0133c3247be06d3c21c0b,ce0718d609ae0f06031490c462fff571210d0d0c,82385a56f25847fe6966d46bb58f302c4597953c,7f1bc20c9c21d1751f25e2ed5dbfda28e1b2554a,3fc1c8fa39c6996247670b3a9182b320dc3d9f0a,5baf7217c2d6e9596923d5e4349928f50c1a6d40,10983f664fc2be47f37d3bef4c78e2f088620fef,a35c02b945625781267821b43bd2eb609616ce65,26f99f7138f8721213e77ac21d013d4fa1716ef0,358dc9545d5faa51c88d04edf8c902068e13b106,3688b1d72f939347de1233defbb4f7ae706eb5ef,e62e836b877d4189e890fcc9d0d4d7d09964933a,4608951935961c98acdd8a88cb4d3dfd0358fa6c,9ad1d8c3a29b04ff542c90d2f6e05eaeddc42a97,d6a858af60bc6f39894eb9118b1aa44f8cfd6be2,...,3160e934656e963e9e0043b2f4f90e2648bf743d,8ca53a415b0219e5aa117e7e4a36d119352e4001,065fdf6b71a04ac9f116ef6c48d5ad11aa4f81d2,1e6cf70bccf9f19d88b72c8d626704099aaed9c9,6ec546bda8ada6afc76e5d5c5d6a4c7c3fce1f32,92fa127222645a92ea37393b7607e90945faf15d,215392414424d604bb113570ebd2f9da75b4761e,7a539d7ef9c0a69fd5dd0b09e98aeee85ec0194b,944b1b3f0749817e0815937b4f5efa5f719c50a2,9fa437dcd243128911d16094767209c2d04947cb,0df5ea55a86bf40b688994f1372d23ca1c7933d6,f71bfb2d30a2d98762544abce3534422d2fe99c2,a62958e9857892a6acb4d98344d93dd69a550a3e,2f81febb1050d0a35d9e2b66b724160b7fbe47a0,a43581283d31c0eae988b34ec85ca39000c680cd,c87c9fe93ddb4dc25d1d241c87a340ecf5be19f5,f9402fa4c19b3124c5a395f2bf443e0417dd0380,6ec256be9ad01a967e5eac307c10fd4d319d6b72,730d8d58fc114ba448f60535d7843c49feec7578,0f7d10d5fabcc4d22bcae8ee90e4f09d42932415,9058e469ff561b485e22afa134d49679818bf80f,b30b484a4b6aedee66258c6abe85cd4d1ecfbd9e,9c027992989747bb680bf798c91130ed4d1b8d65,67ade87897d10497a8b9e774c2cd5c3e821783db,1bb5d0affd5b295b22c204ce1b092259299b3f72,984bd7dc2a680eaaf85955f278ecbcc9cacfe36c,9537a11721507dd679188a0f165abdc43a59fae5,d479f2fe67b64cb4c00a1a547ab12dd99609824e,966c43ea0250eba77ec615eccfe18625850d65af,cda785a670b1dfb74431b92a4dff4c4eebb612e9,fb6f12c3dadd71dd9b0bb928205ad07725ab405a,adb7b69f9b8c690c9a0c8b81126f653cb6f32c72,305a7cba0b9e5c983ce179bfebfe72a92a9a66c7,e229e5b52a293e7e6f5112dd6c8f6be40789fe3f,8299cd39d69e1aec8f37ed2f895be4b9997e7a56,c26c54c0dfa013b4ee1e15a3f11bcfa8215c0a99,94f0be14f76dfbb65a8d9c920542f06e4f79fa26,a630919f315899ebf8650eb07d1097204ba5a2d3,6471b2899033a43f16bfd62b8198ef9557a49f62,1ad3144a286d2b32af03f8e275191f8c48c77e10
0,1913,141.92305,82.461280,-1,0.047978,2.039591,-1.667219,-0.036449,1.770552,-3.012047,3.039011,2.449272,0.084944,-0.073939,-4.887742,0.601928,0.313062,0.954065,1.892878,-0.495190,1340,0.087030,0.498510,-47,-52,-54,-54,-54,-54,-59,-61,-66,-69,-76,-77,-77,-77,-77,-77,-78,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
1,3812,141.92305,82.461280,-1,0.047978,2.039591,-1.667219,-0.036449,1.770552,-3.012047,3.039011,2.449272,0.084944,-0.073939,-4.887742,0.601928,0.313062,0.954065,1.892878,-0.495190,1340,0.087030,0.498510,-67,-55,-68,-54,-54,-54,-56,-86,-66,-74,-76,-86,-86,-77,-77,-77,-83,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
2,3120,142.00758,87.245865,-1,-1.073788,2.274152,-3.540803,-0.402787,0.726523,-2.461220,2.482587,2.711894,-0.021243,-0.207090,-1.224007,-0.229758,0.409849,0.800017,1.428015,-0.252707,1138,0.237318,0.391589,-65,-67,-71,-71,-54,-71,-56,-86,-66,-78,-76,-85,-86,-77,-77,-79,-85,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
3,1225,

,wps_diff,x,y,floor_int,acce_x_avg,acce_y_avg,acce_z_avg,magn_x_avg,magn_y_avg,magn_z_avg,magn_st,ahrs_x_avg,ahrs_y_avg,ahrs_z_avg,gyro_x_avg,gyro_y_avg,gyro_z_avg,magn_u_x_avg,magn_u_y_avg,magn_u_z_avg,rel_diff,rel_x,rel_y,eebf5db207eec2f3e041f92153d789270f346821,7805f319f3f591986effe78c5b41143180278f2d,323607d8444900d64151ee06d164738ac727bbce,b26914599f6d9ba16b43975394e1eeb9d82f4bab,02a1be3a5dab38320f879489d8a1e0f2a72768b3,5b225e187d0dec3110683a74d0c9a5a4cb2022f5,d84cce12fbfba61bf930123050f61a11e2a29310,b2546cae6e588d38618eacc557dd0385812197cf,6bc91b3951089c3a225396608b138ca178479924,8464ea586ee5479e1250f938d7c01e9bc68cefe8,b4dbb0b30caa1d0f21b7b4185ba061556cada67a,c48db7f3ed1858bb4fc191230e3d79d5eb178604,d771612396c3e2e557e986fafd9fc2c56a99d3cd,13b7aeaf441f2161481481fe67eace721cff07ab,dbd15d19abe3f9062b14bc403a499a66aa46cf26,c729e2e4f5a2888583cfebcd98b3178023f58b8e,bd9bc0a2092c040bfe6ba12f8aafac24e83b312a,...,ead4eb320daf267710b8ec2f80b572ef14ca55f6,a19dac7dbdfd5b1628917159bb0333d0b4248b01,a5fa56c01bc636432ad0deb09ac769dd8b4bc4c5,c3403babce8901fd7a22f2752ee27e55c93137ef,e8af39b7dfe78bd5413b4af4a667d052e8cbaa77,fb91245a09b92f24ae14fc6e76e1b3549bd6a31d,0697d08b2741a320bcdcbb9624caca0e3a415356,2d199d76f05af1ff3432955b6b52163e8341dda5,b37d43c8c5f1e82765b5a403c19529d89b765d62,0076ff7a084cb2ac8c146139965ab1be296e72c4,b896a2a793aa802db868a7a0fdb29204bde99065,108716739163fd1a6e473359330704f7cd2068f7,e32d9e78dd87486b832f7723c916ca93ad686a54,1ad3144a286d2b32af03f8e275191f8c48c77e10,38f586d6a54a75a4fc9df0325de877b2bce5de6b,097599fa8b0d2bb4426ba0dae5ca9e578cdb8149,dc522df53e28d76a729dc56f81b1295a50ce4289,042e55ae2e3aae88070d507977eb97cd7468ef80,3b61536817d546393995f664b446e4ecf609ed61,295dcf2d00e2d19d02f73fc1d498ed5ca821161c,647649398bea2985feed4728d748939666dfb609,f3d9e7d629e50b8ca7e330d1d7cc11d6a75def8f,c70a6973225a173d7a0f74ca0ff93d07d140ed9d,a9f2d89eb253ffaf137bad7322e7cae41cd575d7,da8b535fe9eb7c6ea9388d1d2c4c80777da0fa2f,315dc4970761f38035d1da2d3592624abf1293ab,cb7c0d32ada93bf41f44aa2b37a5c584e894e040,96524d4078bf26b236a6ccc4efcf8bfb14189ab4,72a8a758661a41895e424613d666109e838b3d5f,af03d645e2ce58455848037f78dc79d962ec95d2,b831c303c1abd40fd8b242f902b6d900bca585af,c87c9fe93ddb4dc25d1d241c87a340ecf5be19f5,6bcdb1157c0d47e8a75d4ae864210156ebc2de96,817b99c65c4d92ab758443590680da2c3129c62b,563c56cbd1eba74106d9d4cf3adef19ddd574858,24f7fe5fa290f803c49bac656c57df514b180b4b,7e642dee5535c87ae5d65d225ee5dab01769b04c,ae2a68d930d314c1c164485b9ac5ef5beb158290,c5de94605289a9fae1869b9c73dde7806c559717,ed61d67ab40541ebb671536a7ea20125119cb964
0,2331,-999.0,-999.0,-999.0,1.064746,1.225558,-1.255399,1.159333,0.514822,-1.047117,1.054114,1.643774,0.634116,0.654989,-0.711122,-0.251695,0.142039,1.664111,1.149071,-0.046599,1135,-0.425353,0.248690,-45,-46,-46,-47,-47,-48,-48,-48,-48,-48,-49,-49,-49,-49,-49,-50,-50,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
1,491,-999.0,-999.0,-999.0,-0.613235,0.763953,-0.653884,1.014568,0.364085,-0.133938,0.131655,0.853575,1.217137,0.662482,-1.282340,-2.855664,-2.339218,1.603389,1.081798,0.356903,187,-0.537325,0.140535,-69,-47,-52,-47,-65,-47,-46,-46,-47,-54,-46,-46,-45,-45,-57,-56,-44,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
2,612,-999.0,-999.0,-999.0,-1.450420,0.826285,-0.507690,-0.235663,1.044081,-1.315001,1.324720,1.649523,0.069557,-0.279638,-0.314469,-0.540203,0.451585,1.078981,1.385278,-0.164967,161,0.295503,0.550062,-69,-42,-59,-41,-58,-42,-42,-42,-41,-55,-45,-47,-46,-47,-53,-52,-46,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
3,824,-999.0,-999